In [1]:
import pandas as pd
import numpy as np
import h5py
import pyBigWig

In [6]:
genome = 'mm10'
window_kb = 5
bin_size = 10

# get promoterome
infile_promoterome=f"/home/jbreda/Promoterome/results/{genome}/promoterome_pm{window_kb}kb_filtered_clustered_sorted.bed"
promoterome = pd.read_csv(infile_promoterome,sep='\t')
idx_minus_strand = (promoterome.strand=='-').values
N_prom = promoterome.shape[0]

# get one tf
my_tf = 'Rara'
infile_svd = f'../results/{genome}/svd/Window_pm{window_kb}kb_bin_size_{bin_size}/{my_tf}.hdf5'
with h5py.File(infile_svd, 'r') as hf:
    u = hf['U'][:,0]
    s = hf['S'][0]
    N_pos = hf['U'].shape[0]//N_prom

chip = np.reshape(u*s,[N_prom,N_pos]).T
# flip promoters on minus strand
chip[:,idx_minus_strand] = chip[:,idx_minus_strand][:,::-1]
#chip[np.isnan(chip)] = 0

In [23]:

# get chrom size
chrom_size_file = '../genome/mm10/Mus_musculus.GRCm38.dna.primary_assembly.chrom_size'
chr_size = []
chr_values = {}
CHR = promoterome.chr.unique()
with open(chrom_size_file,'r') as f:
    for l in f.readlines():
        l = l.strip()
        chr = 'chr'+l.split('\t')[0]
        size = int(l.split('\t')[1])
        if chr in CHR:
            chr_size.append((chr,size))
            chr_values[chr] = np.zeros(int(np.ceil(size/bin_size)))

# fill in chr_values
for p in range(N_prom):
    # skip if all nan
    if np.all(np.isnan( chip[:,p] )):
        continue

    # get absolute and relative idx
    chr = promoterome.at[p,'chr']
    start_idx = int(np.ceil(promoterome.at[p,'start']/bin_size))
    rel_idx =  np.where( ~np.isnan(chip[:,p]) )[0]
    idx = start_idx + rel_idx

    # add values
    chr_values[chr][idx] = chr_values[chr][idx] + chip[rel_idx,p]

# write bigwig
with pyBigWig.open('../results/mm10/svd/Window_pm5kb_bin_size_10/Rara.bw', 'w') as bw:

    # add headers
    bw.addHeader(chr_size)

    # fill in the big wig chr by chr
    for chr in chr_values.keys():
        print(chr)
        
        # get indices with non zero values
        idx = np.where( chr_values[chr] != 0 )[0]

        # get coordinates and values
        chrs = idx.shape[0]*[chr]
        starts = idx*bin_size
        ends = starts + bin_size
        values = chr_values[chr][idx]

        # add entries
        bw.addEntries(chrs, starts, ends, values)


chr1
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chrX
chrY
